In [ ]:
import yfinance as yf
import numpy as np

from gudhi.datasets.generators import points
from gudhi import SimplexTree

import plotly.graph_objects as go
from scipy.spatial.distance import cdist

In [ ]:
circle_sample = points.sphere(n_samples=500, ambient_dim=2, radius=1, sample="random")

fig = go.Figure(
  data=go.Scattergl(
    x=circle_sample[:,0],
    y=circle_sample[:,1],
    mode='markers'
  ),
  layout=go.Layout(
    width=1000,
    height=1000,
    paper_bgcolor="#FFFFFF",
    plot_bgcolor='#FFFFFF'
  )
)

fig.update_xaxes(showgrid=False, zeroline=False, range=[-2.,2.], tickvals=[])
fig.update_yaxes(showgrid=False, zeroline=False, range=[-2.,2.], tickvals=[])

fig.show()

In [ ]:
num_vertices=10

# --- build the complex (yours) ------------------------------------------------
point_array = np.random.uniform(-1,1,size=(num_vertices,3))
distance_matrix = cdist(point_array, point_array)
filtered_complex = SimplexTree.create_from_array(distance_matrix, max_filtration=2/(num_vertices**(1./3.)))
filtered_complex.expansion(3)

# --- pull filtration once, then split by simplex dimension --------------------
filtration = list(filtered_complex.get_filtration())

# --- collect once, split, and sort by filtration -----------------------------
filtration = list(filtered_complex.get_filtration())

edge_idx = np.array([ (s[0][0], s[0][1]) for s in filtration if len(s[0]) == 2 ], dtype=int)
edge_filtration = np.array([ s[1] for s in filtration if len(s[0]) == 2 ], dtype=float)

tri_idx  = np.array([ (s[0][0], s[0][1], s[0][2]) for s in filtration if len(s[0]) == 3 ], dtype=int)
face_filtration = np.array([ s[1] for s in filtration if len(s[0]) == 3 ], dtype=float)

# thresholds = every unique birth time (edges + faces) ∪ {0}
raw_births = np.concatenate(([0.0], edge_filtration, face_filtration)).astype(float)
thresholds = np.sort(np.unique(np.round(raw_births, 12)+0.0001))  # round to kill float fuzz

# sort edges/faces by filtration to enable fast slicing via searchsorted
e_order = np.argsort(edge_filtration)
f_order = np.argsort(face_filtration)
edge_idx_sorted = edge_idx[e_order]
edge_filtration_sorted = edge_filtration[e_order]
tri_idx_sorted = tri_idx[f_order]
face_filtration_sorted = face_filtration[f_order]

# helper: pack first k edges as a single polyline (None-separated)
def edge_lines_upto(k: int):
    if k <= 0:
        return [], [], []
    pairs = edge_idx_sorted[:k]
    xs, ys, zs = [], [], []
    for a, b in pairs:
        xs += [point_array[a,0], point_array[b,0], None]
        ys += [point_array[a,1], point_array[b,1], None]
        zs += [point_array[a,2], point_array[b,2], None]
    return xs, ys, zs

# initial filtration at the first threshold (usually 0)
t0 = thresholds[0]
kE0 = int(np.searchsorted(edge_filtration_sorted, t0, side="right"))
kF0 = int(np.searchsorted(face_filtration_sorted, t0, side="right"))

# --- traces (points, edges, faces) -------------------------------------------
fig = go.Figure(layout=go.Layout(width=1000, height=1000))
fig.add_trace(go.Scatter3d(
    x=point_array[:,0], y=point_array[:,1], z=point_array[:,2],
    mode='markers', hoverinfo='none',
    marker=dict(size=5, color="#FFFFFF"),
    name="points", showlegend=False
))

ex0, ey0, ez0 = edge_lines_upto(kE0)
fig.add_trace(go.Scatter3d(
    x=ex0, y=ey0, z=ez0, mode='lines', hoverinfo='none',
    line=dict(width=4, color="#AD8934"),
    name="edges",
))

fig.add_trace(go.Mesh3d(
    x=point_array[:,0], y=point_array[:,1], z=point_array[:,2],
    i=tri_idx_sorted[:kF0,0] if kF0>0 else np.array([], dtype=int),
    j=tri_idx_sorted[:kF0,1] if kF0>0 else np.array([], dtype=int),
    k=tri_idx_sorted[:kF0,2] if kF0>0 else np.array([], dtype=int),
    color="#0DFF5D", hoverinfo='none',
    showscale=False, 
    opacity=0.35,
    name="faces"
))

frame_names = [f"{t:.12f}" for t in thresholds]

anim_args = {"mode": "immediate",
             "frame": {"duration": 0, "redraw": True},
             "transition": {"duration": 0}}

# --- Build frames; each frame rewires +/− to its own neighbors ---------------
frames = []
for i, t in enumerate(thresholds):
    kE = int(np.searchsorted(edge_filtration_sorted, t, side="right"))
    kF = int(np.searchsorted(face_filtration_sorted, t, side="right"))
    ex, ey, ez = edge_lines_upto(kE)

    frames.append(go.Frame(
        name=frame_names[i],
        data=[
            go.Scatter3d(x=ex, y=ey, z=ez, showlegend=False, hoverinfo='none'),  # edges (trace index 1)
            go.Mesh3d(
                i=tri_idx_sorted[:kF,0] if kF>0 else np.array([], int),
                j=tri_idx_sorted[:kF,1] if kF>0 else np.array([], int),
                k=tri_idx_sorted[:kF,2] if kF>0 else np.array([], int),
                color="#0DFF5D", hoverinfo='none',
                showscale=False, 
                opacity=0.35,
                name="faces"
            )
        ],
        traces=[1, 2],
        layout=go.Layout(
            updatemenus=[dict(
                type="buttons", direction="left", showactive=False,
                x=0.5, y=1.08, xanchor="center", yanchor="top",
                buttons=[
                    dict(label="<<<", method="animate", args=[[frame_names[0]], anim_args]),
                    dict(label="<<", method="animate", args=[[frame_names[max(0, i-5)]], anim_args]),
                    dict(label="<", method="animate", args=[[frame_names[max(0, i-1)]], anim_args]),
                    dict(label=">", method="animate", args=[[frame_names[min(i+1, len(frame_names)-1)]], anim_args]),
                    dict(label=">>", method="animate", args=[[frame_names[min(i+5, len(frame_names)-1)]], anim_args]),
                    dict(label=">>>", method="animate", args=[[frame_names[-1]], anim_args])
                ]
            )]
        )
    ))

fig.frames = frames

camera = dict(up=dict(x=0, y=0, z=1), center=dict(x=0, y=0, z=0), eye=dict(x=.25, y=.25, z=.25))

# Initial buttons for the very first view
first_next = frame_names[1] if len(frame_names) > 1 else frame_names[0]
fig.update_layout(
    updatemenus=[dict(
        type="buttons", direction="left", showactive=False,
        x=0.5, y=1.08, xanchor="center", yanchor="top",
        buttons=[
            dict(label="<<<", method="animate", args=[[frame_names[0]], anim_args]),
            dict(label="<<", method="animate", args=[[frame_names[0]], anim_args]),
            dict(label="<", method="animate", args=[[frame_names[0]], anim_args]),
            dict(label=">", method="animate", args=[[first_next], anim_args]),
            dict(label=">>", method="animate", args=[[frame_names[min(5, len(frame_names)-1)]], anim_args]),
            dict(label=">>>", method="animate", args=[[frame_names[-1]], anim_args])
        ]
    )],
    scene=dict(
        uirevision=True,  
        xaxis=dict(showbackground=True, showticklabels=False, zeroline=False, title="", range=[-5, 5], backgroundcolor="#0f1117", gridcolor='#0f1117'),
        yaxis=dict(showbackground=True, showticklabels=False, zeroline=False, title="", range=[-5, 5], backgroundcolor="#0f1117", gridcolor='#0f1117'),
        zaxis=dict(showbackground=True, showticklabels=False, zeroline=False, title="", range=[-5, 5], backgroundcolor="#0f1117", gridcolor='#0f1117'),
        camera=camera,
    ),
    margin=dict(l=0.0, r=0.0, b=0.0, t=0.0),
    paper_bgcolor="#0f1117"
)

fig.show()
